In [1]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "testing"))
session = driver.session()

In [7]:
q1 = "match(x) return x limit 5"
for node in session.run(q1):
    print(node)

In [ ]:
# TODO figure out setting up new db
# TODO load sqlite data into neo4j